In [1]:
!pip install -q huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 MB 32.7 MB/s eta 0:00:00:00:0100:01


In [2]:
from datasets import load_dataset

train_ds=load_dataset('csv', data_files='/kaggle/input/pair-dataset/paired_dataset_2025-04-10_00-30-17.csv')['train']
train_ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 2994
})

In [3]:
train_ds[0]

{'anchor': 'R: In April, Natalia sold 48 clips. In May, she sold half as many, which is 48 / 2.\nSo, the total number of clips sold in April and May is 48 + (48 / 2).\n\nA: 72',
 'positive': 'R: In April, Natalia sold 48 clips. In May, she sold half as many, which is 48 / 2.\nSo, the total number of clips sold in April and May is 48 + (48 / 2).\n\nA: 72'}

# Pre-processing dataset

We will split dataset into training, testing and evaluation.

In [4]:
MODEL_NAME='all-MiniLM-L6-v2'
OUTPUT_DIR='encoder-L6-V2/training'
MODEL_DIR='aisuko/encoder-L6-V2'

# Load the Model

In [5]:
# https://www.sbert.net/docs/package_reference/sentence_transformer/losses.html#multiplenegativesrankingloss
import torch
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model=SentenceTransformer(MODEL_NAME).to(device)
model

2025-04-10 05:02:48.846789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744261369.038945      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744261369.090381      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [6]:
from sentence_transformers.losses import MultipleNegativesRankingLoss

loss=MultipleNegativesRankingLoss(model)

In [7]:
from sentence_transformers import SentenceTransformerTrainingArguments
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers import SentenceTransformerTrainer

train_args=SentenceTransformerTrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    # per_device_eval_batch_size=16,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=True,
    bf16=False,
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # losses that use "in-batch negatives" benefit from no duplicates
    # eval_strategy='steps',
    # eval_steps=100,
    save_strategy='steps',
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    report_to='tensorboard',
    run_name='encoder-L6-V2'
)

trainer=SentenceTransformerTrainer(
    model=model,
    args=train_args,
    train_dataset=train_ds,
    loss=loss
)

trainer.train()

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
100,0.000100


TrainOutput(global_step=188, training_loss=0.000744679511742706, metrics={'train_runtime': 20.0565, 'train_samples_per_second': 149.278, 'train_steps_per_second': 9.374, 'total_flos': 0.0, 'train_loss': 0.000744679511742706, 'epoch': 1.0})

In [8]:
model.save_pretrained(MODEL_DIR)

In [9]:
import kagglehub

VARIATION_SLUG='v0.1.0'

handler = f'aisuko/encoder-L6-V2/transformers/{VARIATION_SLUG}'

kagglehub.model_upload(
    handle=handler,
    local_model_dir=MODEL_DIR,
    license_name='MIT',
    version_notes='v0.1.0'
)

Uploading Model https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.0 ...
Starting upload for file aisuko/encoder-L6-V2/sentence_bert_config.json


Uploading: 100%|██████████| 53.0/53.0 [00:00<00:00, 186B/s]

Upload successful: aisuko/encoder-L6-V2/sentence_bert_config.json (53B)
Starting upload for file aisuko/encoder-L6-V2/tokenizer.json



Uploading: 100%|██████████| 712k/712k [00:00<00:00, 3.80MB/s]

Upload successful: aisuko/encoder-L6-V2/tokenizer.json (695KB)
Starting upload for file aisuko/encoder-L6-V2/config.json


...


Uploading: 100%|██████████| 349/349 [00:00<00:00, 1.86kB/s]

Upload successful: aisuko/encoder-L6-V2/modules.json (349B)
Starting upload for file aisuko/encoder-L6-V2/1_Pooling/config.json



Uploading: 100%|██████████| 296/296 [00:00<00:00, 1.62kB/s]

Upload successful: aisuko/encoder-L6-V2/1_Pooling/config.json (296B)


Your model instance version has been created.
Files are being processed...
See at: https://www.kaggle.com/models/aisuko/encoder-L6-V2/transformers/v0.1.0
